In [314]:
from collections import defaultdict
import math
import nltk
from nltk.lm.models import Lidstone
from nltk.lm.models import Laplace
from nltk.corpus import machado
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
import re


# 1 - Para o trecho do poema "Ainda que mal"
"ainda que mal pergunte
ainda que mal respondas
ainda que mal te entenda
ainda que mal repitas"


In [413]:
def generate_ngrams(text, n):
    words = text.split()
    bigrams = []
    for i in range(len(words) - n + 1):
        bigram =  " ".join(words[i:i+n]) 
        bigrams.append(tuple(bigram.split())) 
    
    return bigrams

def probabilidade_verso( text,model, mostrar=False):
    # Sequência de palavras
    if type(text) == str:
        sequence = tuple(text.split())
    else:
        sequence = text
    
    # Inicialize a probabilidade comum com 1.0 (inicialmente, multiplicamos por 1)
    common_probability = 1.0
    
    # Calcular a probabilidade comum
    for i in range(len(sequence) - 1):
        gram = sequence[i]
        #if mostrar:print(sequence[i])
        print(gram )
        if gram in model.vocab :
            common_probability *= model.score(gram)
        else:
            # Se o bigrama não estiver no corpus, defina a probabilidade como 0
            common_probability = 0.0
            break  # Saia do loop se encontrar um bigrama ausente
    
    # Imprimir a probabilidade comum
    if mostrar: print(f'Probabilidade Comum da Sequência: {common_probability:.10f}')
    return common_probability

def entropia(prob):
    return prob * math.log2(prob)

def perplexidade(frase,model):

    common_probability = probabilidade_verso(frase,model, False)
    
    perplexity = 1
    
    if common_probability != 0.0:
        perplexity = pow(2, entropia(common_probability))
        
    print(f'Perplexidade do Modelo: {perplexity:.4f}')

In [313]:
text = ("ainda que mal pergunte "
        "ainda que mal respondas "
        "ainda que mal te entenda "
        "ainda que mal repitas")

a) Ache todos os bigramas do trecho

In [364]:
bi = generate_ngrams(text, 2)
print(bi)

[('ainda', 'que'), ('que', 'mal'), ('mal', 'pergunte'), ('pergunte', 'ainda'), ('ainda', 'que'), ('que', 'mal'), ('mal', 'respondas'), ('respondas', 'ainda'), ('ainda', 'que'), ('que', 'mal'), ('mal', 'te'), ('te', 'entenda'), ('entenda', 'ainda'), ('ainda', 'que'), ('que', 'mal'), ('mal', 'repitas')]


b)Calcule a probabilidade logarítmica de cada bigrama encontrado

In [381]:
tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(text)]
n = 2
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
model = MLE(n)
model.fit(train_data, padded_sents)

for i in bi:
    print(f'Probabilidade de {i}: {probabilidade_verso(i,model)}')

Probabilidade de ('ainda', 'que'): 0.21052631578947367
Probabilidade de ('que', 'mal'): 0.21052631578947367
Probabilidade de ('mal', 'pergunte'): 0.21052631578947367
Probabilidade de ('pergunte', 'ainda'): 0.05263157894736842
Probabilidade de ('ainda', 'que'): 0.21052631578947367
Probabilidade de ('que', 'mal'): 0.21052631578947367
Probabilidade de ('mal', 'respondas'): 0.21052631578947367
Probabilidade de ('respondas', 'ainda'): 0.05263157894736842
Probabilidade de ('ainda', 'que'): 0.21052631578947367
Probabilidade de ('que', 'mal'): 0.21052631578947367
Probabilidade de ('mal', 'te'): 0.21052631578947367
Probabilidade de ('te', 'entenda'): 0.05263157894736842
Probabilidade de ('entenda', 'ainda'): 0.05263157894736842
Probabilidade de ('ainda', 'que'): 0.21052631578947367
Probabilidade de ('que', 'mal'): 0.21052631578947367
Probabilidade de ('mal', 'repitas'): 0.21052631578947367


d) Com o modelo de linguagem generalizado, calcule a probabilidade comum do verso "ainda que mal insista"

In [408]:
probabilidade_verso('ainda que mal insista',model, True)

ainda
que
mal
Probabilidade Comum da Sequência: 0.0093308062


0.009330806239976671

e) Calcule a perplexidade do modelo de linguagem generalizado utilizando o verso "ainda que mal insista” como conjunto de teste.

In [414]:
perplexidade('ainda que mal insista', model)

ainda
que
mal
Perplexidade do Modelo: 0.9573


# 2 Escolha um conjunto de textos  (poemas, letras de música, etc )

In [312]:
def criar_modelo(text,n, model=None, gamma=0.1):
    tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(text)]
    train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
    
    if model == 'li':
        model_machado = Lidstone(gamma,order=n )
    elif model == 'la':
        model_machado = Laplace(order=n)
    else:
        model_machado = MLE(n)
        
    model_machado.fit(train_data, padded_sents)
    return model_machado

def remover_stopwords(texto):
    # Carregue a lista de stopwords da língua desejada (exemplo: português)
    stopwords_lista = nltk.corpus.stopwords.words('portuguese')
    
    # Tokenize o texto em palavras
    palavras = word_tokenize(texto)
    
    # Crie uma lista de palavras sem as stopwords
    palavras_sem_stopwords = [palavra for palavra in palavras if palavra.lower() not in stopwords_lista]
    
    # Recrie o texto sem as stopwords
    texto_sem_stopwords = ' '.join(palavras_sem_stopwords)
    
    return texto_sem_stopwords

def remocao_caracteres_especiais(text):
    padrao = r'[^\w\s]?\n'
    return re.sub(padrao, ' ', text)

def trans_corpus_text(corpus):
    text = ''
    for i in corpus:
        text += machado.raw(i) +' '
    
    #Pre-processamento
    #text = remover_stopwords(text)
    text = remocao_caracteres_especiais(text)
    
    return text



a) Separe 10% dos textos como conjunto de testes e e o restante como conjunto de treinamento

In [243]:
corpus = []
separa_treino = 0.1

for i in machado.fileids():
        corpus.append(i)

tam_treinamento = math.ceil(len(corpus) * separa_treino)
corpus_teste = []

for i, t in enumerate(corpus):
    if i == tam_treinamento:break
    corpus_teste.append(t)
    corpus.remove(t)

print('Treinamento = ',len(corpus))
print('Teste = ', len(corpus_teste))

Treinamento =  221
Teste =  25


Transforma o conjunto de teste me uma lista de n-grams

In [297]:
ugt = generate_ngrams(trans_corpus_text(corpus_teste),1)
bgt = generate_ngrams(trans_corpus_text(corpus_teste),2)
tgt = generate_ngrams(trans_corpus_text(corpus_teste),3)

b) Construa um modelo de linguagen  explorando diferentes n-gramas (unigrama, bigrama e trigrama) e métodos de suavização (Laplace vs Lidstone)
c) Faça uma análise do melhor modelo de linguagem com base na sua perplexidade no conjunto de teste

LIDSTONE

In [263]:
model_unigram_li = criar_modelo(trans_corpus_text(corpus), 1, model='li')
model_bigram_li  = criar_modelo(trans_corpus_text(corpus), 2, model='li')
model_trigram_li  = criar_modelo(trans_corpus_text(corpus), 3, model='li')

In [298]:
print('Perplexicidade unigram Lidstone: ',  model_unigram_li.perplexity(ugt)) 
print('Perplexicidade bigram Lidstone: ', model_bigram_li.perplexity(bgt))
print('Perplexicidade trigram Lidstone: ', model_trigram_li.perplexity(tgt))

Perplexicidade unigram Lidstone:  10894.325853126034
Perplexicidade bigram Lidstone:  8572.914977396873
Perplexicidade trigram Lidstone:  38011.19129848849


LAPLACE

In [205]:
model_unigram_la = criar_modelo(trans_corpus_text(corpus), 1, model='la')
model_bigram_la = criar_modelo(trans_corpus_text(corpus), 2,model='la')
model_trigram_la = criar_modelo(trans_corpus_text(corpus), 3,model='la')

In [303]:
print('Perplexicidade unigram Laplace: ',  model_unigram_la.perplexity(ugt) ) 
print('Perplexicidade bigram Laplace: ', model_bigram_la.perplexity(bgt))
print('Perplexicidade trigram Laplace: ', model_trigram_la.perplexity(tgt))

Perplexicidade unigram Laplace:  7069.282189140654
Perplexicidade bigram Laplace:  14897.096413179352
Perplexicidade trigram Laplace:  52537.56758533888


d) Escolha o melhor modelo e gere um texto de até 30 tokens predizendo a proxima palavra de maxima probabilidade.

In [424]:
en = 'amor que vem'
t = f'{en} '
for i in model_trigram_la.generate(30, text_seed=en.split()):
    t += i + ' '
print(t)

amor que vem isto pacheco eu não posso perder um compasso que abria as velas rasgadas e minuciosamente examinadas os infelizes e entulhar o buraco estava tapado mas não havendo ninguém imaginei que 
